# Homework 2

In [7]:
import numpy as np
from fastembed import TextEmbedding

## Q1. Embedding the query

In [16]:
query = "I just discovered the course. Can I join now?"
model = "jinaai/jina-embeddings-v2-small-en"

embedding = TextEmbedding(model_name=model)

embedded_query = embedding.embed(query)

In [18]:
embedded_query = list(embedded_query)[0]

In [21]:
embedded_query.min()

np.float64(-0.11726373885183883)

## Q2. Cosine similarity with another vector

In [22]:
np.linalg.norm(embedded_query)

np.float64(1.0)

In [23]:
embedded_query.dot(embedded_query)

np.float64(1.0000000000000002)

In [24]:
doc = "Can I still join the course after the start date?"
embedded_doc = embedding.embed(doc)

embedded_doc = list(embedded_doc)[0]

In [28]:
np.linalg.norm(embedded_doc)

np.float64(1.0)

In [30]:
cosine_sim = embedded_doc.dot(embedded_query)

cosine_sim

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

In [32]:
documents = [
    {
        "text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
        "section": "General course-related questions",
        "question": "Course - Can I still join the course after the start date?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.",
        "section": "General course-related questions",
        "question": "Course - Can I follow the course after it finishes?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.",
        "section": "General course-related questions",
        "question": "Course - What can I do before the course starts?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.",
        "section": "General course-related questions",
        "question": "How can we contribute to the course?",
        "course": "data-engineering-zoomcamp",
    },
]

In [ ]:
embedded_texts = embedding.embed([document['text'] for document in documents])
embedded_texts = list(embedded_texts)

In [ ]:
cosine_sims = np.array(embedded_texts).dot(embedded_query)
cosine_sims.argmax()

np.int64(1)

## Q4. Ranking by cosine, version two

In [55]:
full_text = [doc["question"] + " " + doc["text"] for doc in documents]

embedded_texts = embedding.embed(full_text)
embedded_texts = list(embedded_texts)

In [ ]:
cosine_sims = np.array(embedded_texts).dot(embedded_query)

cosine_sims.argmax()

np.int64(0)

In [57]:
query

'I just discovered the course. Can I join now?'

In [59]:
documents[0]["question"] + " " + documents[0]["text"]

"Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [60]:
documents[1]["text"]

'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.'

The score will be different, because we calculate cosine similarity between the query with:
- text (answer) only for version one,
- question with text (answer) for version two.

## Q5. Selecting the embedding model

The smallest embedding dimensionality available in FastEmbed models is 384. BAAI/bge-small-en has dimension of 384. 

## Q6. Indexing with qdrant 

In [62]:
from qdrant_client import QdrantClient, models

In [61]:
import requests

docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course["course"]
    if course_name != "machine-learning-zoomcamp":
        continue

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)


In [63]:
qd_client = QdrantClient("http://localhost:6333")

In [64]:
model = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384

In [65]:
collection_name = "zoomcamp-faq"
qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [66]:
qd_client.create_payload_index(
    collection_name=collection_name, field_name="course", field_schema="keyword"
)


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [67]:
points = []

for i, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"]
    vector = models.Document(text=text, model=model)
    point = models.PointStruct(id=i, vector=vector, payload=doc)
    points.append(point)

In [68]:
qd_client.upsert(collection_name=collection_name, points=points)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]d:\Project\llm-zoomcamp\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ariaw\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 files: 100%|██████████| 5/5 [01:04<00:00,

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [70]:
def vector_search(question):
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model 
        ),
        limit=5,
        with_payload=True
    )
    return query_points

In [75]:
scores = []
for point in vector_search(query).points:
    scores.append(point.score)

In [76]:
max(scores)

0.87031734